In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

     |████████████████████████████████| 8.0 MB 4.2 MB/s 
     |████████████████████████████████| 2.9 MB 4.3 MB/s 
     |████████████████████████████████| 376 kB 4.3 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 


In [ ]:
gpu=0

In [ ]:
edge_list = np.array([(4,3),(3,5),(5,6),(6,7),(7,8),(3,9),(9,10),
                 (10,11),(11,12),(3,2),(2,1),(1,13),(1,17),
                 (13,14),(14,15),(15,16),(17,18),(18,19),
                 (19,20)]) - 1
l1,l2 = [],[]
for i,j in edge_list:
    l1.append(i)
    l1.append(j)
    l2.append(j)
    l2.append(i)

edge_index = torch.tensor([l1,l2], dtype=torch.long)
if gpu is not None:
    edge_index = edge_index.cuda(gpu)
print(edge_index.size())

torch.Size([2, 38])


In [ ]:
from torch_geometric.nn import GCNConv
def bn_init(bn, scale):
    nn.init.constant_(bn.weight, scale)
    nn.init.constant_(bn.bias, 0)

class gcn(nn.Module):
    def __init__(self, in_C, out_C):
        super(gcn,self).__init__()
        self.in_C = in_C
        self.bn = nn.BatchNorm1d(in_C)
        self.dropout = nn.Dropout(0.3)
        self.conv1 = GCNConv(in_C, 32)
        self.conv2 = GCNConv(32, 64)
        self.conv3 = GCNConv(64, out_C)
    def forward(self, x, edges):
        # input  N  V  C_in
        # output N  V*C_out
        # Batch normalization
        N, V, C = x.size()
        x = x.permute(0,2,1).contiguous().view(N,C,V)
        x = self.bn(x)
        x = x.permute(0,2,1).contiguous().view(N, V, C)

        # Graph convolution with residual
        x = self.conv1(x, edges)
        x = F.relu(x)
        #residual = x

        x = self.conv2(x, edges)
        x = F.relu(x)
        x = self.conv3(x, edges)
        x = F.relu(x)
        #x = x + residual

        N, V, C = x.size()
        x = x.view(N, V*C).contiguous()
        return self.dropout(x)

class gcn_LSTM(nn.Module):
    def __init__(self, gcn_in_C, n_classes,  gcn_out_C=64, lstm_C=256, lstm_n_layer=2, T=16, V=20):
        super(gcn_LSTM, self).__init__()
        # input N, T, V, C
        # output N, n_classes
        self.T = T
        self.V = V
        self.gcn_layers=nn.ModuleList([gcn(gcn_in_C, gcn_out_C) for i in range(T)])
        #self.gcn_layer = gcn(gcn_in_C, gcn_out_C)
        self.lstm1 = nn.LSTM(gcn_out_C*V, lstm_C, lstm_n_layer, batch_first=True)
        self.classifier = nn.Linear(lstm_C, n_classes)

    def forward(self, x, edges):
        N, T, V, C = x.size()
        assert V==self.V
        assert T==self.T

        output = torch.tensor([])
        if gpu is not None:
            output = output.cuda(gpu)
        for i in range(T):
            output_t = self.gcn_layers[i](x[:, i, :, :],edges)
            #output_t = self.gcn_layer(x[:, i, :, :],edges)
            output_t = output_t.unsqueeze(1)
            if gpu is not None:
                output_t = output_t.cuda(gpu)
            output = torch.cat((output, output_t ), 1)
        output, (ht,ct)= self.lstm1(output)

        output = self.classifier(ht[-1])

        return output


In [ ]:
def random_manipulateX(x):
    # T,V,C
    x[:,:,0] += (torch.rand(1)-0.5)*10
    x[:,:,1] += (torch.rand(1)-0.5)*10
    x[:,:,2] += (torch.rand(1)-0.5)*10
    flip_x, flip_y, flip_z = torch.rand(1)>0.5, torch.rand(1)>0.5,torch.rand(1)>0.5
    if flip_x:
        x[:,:,0] = -x[:,:,0]
    if flip_y:
        x[:,:,1] = -x[:,:,1]
    if flip_z:
        x[:,:,2] = -x[:,:,2]
    return x

class GCNDataset(Dataset):
    def __init__(self,filename,hasLabel=True,aug = False,balance = 0,frac=1):
        self.df = pd.read_csv(filename,header=None)
        self.length = len(self.df)
        self.aug=aug
        if hasLabel:
            if balance>0:
                self.df['freq']=self.df.groupby(961)[961].transform('count')
                self.df['freq'] = sum(self.df['freq'])/(self.df['freq']**balance)
                self.df = self.df.sample(frac=frac,weights=self.df.freq,replace=False).reset_index(drop=True)
                self.X = torch.tensor(self.df.iloc[:,1:-2].values.astype('float32'))
                self.labels = self.df.iloc[:,-2].values.astype('int32')-1
            else:
                self.df = self.df.sample(frac=frac,replace=False).reset_index(drop=True)
                self.X = torch.tensor(self.df.iloc[:,1:-1].values.astype('float32'))
                self.labels = self.df.iloc[:,-1].values.astype('int32')-1
            # N, C, V, T
            #self.X = self.X.reshape((self.length ,16, 20, 3)).permute(0,3,2,1).contiguous()
            # N, T, V, C
            self.X = self.X.reshape((int(self.length*frac) ,16, 20, 3)).contiguous()
            self.Y = torch.tensor(self.labels,dtype=torch.long)
        else:
            self.X = torch.tensor(self.df.iloc[:,1:].values.astype('float32'))
            self.X = self.X.reshape((self.length ,16, 20, 3)).contiguous()
            self.Y=torch.tensor(np.zeros(self.length),dtype=torch.long)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        x = self.X[index]
        if self.aug:
            x = random_manipulateX(x)
        y = self.Y[index]
        return x, y 

In [ ]:
root_path = "drive/MyDrive/SML/"
bs = 64
train_set = GCNDataset(filename=root_path+"training_set.csv",aug=False)
val_set = GCNDataset(filename=root_path+"val_set.csv")
test_set = GCNDataset(filename=root_path+"test.csv",hasLabel=False)
train_loader = DataLoader(train_set,batch_size=bs,shuffle=True)
val_loader = DataLoader(val_set,batch_size=bs)
test_loader = DataLoader(test_set,batch_size=bs)

In [ ]:
net = gcn_LSTM(3,49)
#net.load_state_dict(torch.load(root_path+'gcn_LSTM_best.pkl'))

gpu = 0 #gpu ID
net.cuda(gpu)


criterion = nn.CrossEntropyLoss()
opti = optim.Adam(net.parameters(), lr = 0.001,weight_decay=0.0005)
#opti.load_state_dict(torch.load(root_path+'gcn_LSTM_best_optim.pkl'))

print()

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(net)

2216337

In [ ]:
net

gcn_LSTM(
  (gcn_layers): ModuleList(
    (0): gcn(
      (bn): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.3, inplace=False)
      (conv1): GCNConv(3, 32)
      (conv2): GCNConv(32, 64)
      (conv3): GCNConv(64, 64)
    )
    (1): gcn(
      (bn): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.3, inplace=False)
      (conv1): GCNConv(3, 32)
      (conv2): GCNConv(32, 64)
      (conv3): GCNConv(64, 64)
    )
    (2): gcn(
      (bn): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.3, inplace=False)
      (conv1): GCNConv(3, 32)
      (conv2): GCNConv(32, 64)
      (conv3): GCNConv(64, 64)
    )
    (3): gcn(
      (bn): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.3, inplace=False)
      (conv1): GCNConv(3, 32)
      (conv2): GCNConv(32, 64)

In [ ]:
def adjust_learning_rate(lr,wd=0.001):
    for param_group in opti.param_groups:
        param_group['lr'] = lr
        param_group['weight_decay']=wd
    return lr

def accuracy(logit,target):
    a=(torch.argmax(logit,dim=1)==target).sum()
    return a

def evaluate(model, criterion, dataloader, gpu):
    model.eval()
    acc = 0
    count = 0
    with torch.no_grad():
        for i,(x,y) in enumerate(dataloader):
            x,y = x.cuda(gpu), y.cuda(gpu)
            logits = model(x, edge_index)
            acc+= accuracy(logits, y)
            count += bs

    return acc / count

def train():
    best_acc=0.415
    best_epoch = 0
    for epoch in range(150):
        total = 0
        correct = 0
        """
        if epoch==0:
            adjust_learning_rate(0.01)
        elif epoch==50:
            adjust_learning_rate(0.001)
        elif epoch==120:
            adjust_learning_rate(0.0001)
        elif epoch==140:
            adjust_learning_rate(0.00001)
        """
        for i, (x,y) in enumerate(train_loader):
            net.train()
            opti.zero_grad()
            x,y=x.cuda(gpu),y.cuda(gpu)
            opti.zero_grad()
            logit = net(x,edge_index)
            loss = criterion(logit,y)
            loss.backward()
            opti.step()
            correct+=accuracy(logit,y)
            total+=bs

        dev_acc = evaluate(net, criterion, val_loader, gpu)
        if dev_acc>best_acc+0.003:
            best_acc=dev_acc
            torch.save(net.state_dict(), root_path+'gcn_LSTM_best.pkl')
            torch.save(opti.state_dict(), root_path+"gcn_LSTM_best_optim.pkl")
        print("epoch",epoch,"train acc:",round(float(correct/total),5),"dev_acc:",round(float(dev_acc),5))
train()

epoch 0 train acc: 0.15704 dev_acc: 0.2276
epoch 1 train acc: 0.24656 dev_acc: 0.26146
epoch 2 train acc: 0.29237 dev_acc: 0.30365
epoch 3 train acc: 0.32693 dev_acc: 0.31458
epoch 4 train acc: 0.34931 dev_acc: 0.33333
epoch 5 train acc: 0.36361 dev_acc: 0.33594
epoch 6 train acc: 0.37248 dev_acc: 0.3599
epoch 7 train acc: 0.3795 dev_acc: 0.36615
epoch 8 train acc: 0.39314 dev_acc: 0.36354
epoch 9 train acc: 0.39897 dev_acc: 0.38698
epoch 10 train acc: 0.40307 dev_acc: 0.37708
epoch 11 train acc: 0.41327 dev_acc: 0.38594
epoch 12 train acc: 0.42624 dev_acc: 0.39167
epoch 13 train acc: 0.43988 dev_acc: 0.40052
epoch 14 train acc: 0.42797 dev_acc: 0.38594
epoch 15 train acc: 0.44147 dev_acc: 0.39844
epoch 16 train acc: 0.4465 dev_acc: 0.39948
epoch 17 train acc: 0.45445 dev_acc: 0.40365
epoch 18 train acc: 0.45339 dev_acc: 0.39948
epoch 19 train acc: 0.45379 dev_acc: 0.40313
epoch 20 train acc: 0.47378 dev_acc: 0.39271
epoch 21 train acc: 0.47259 dev_acc: 0.41042
epoch 22 train acc: 0.47

KeyboardInterrupt: ignored

In [ ]:
net.load_state_dict(torch.load(root_path+'gcn_LSTM_best.pkl'))
def predict(net,dataloader):
    net.eval()
    predictions = torch.tensor([]).cuda(gpu)
    logits = torch.tensor([]).cuda(gpu)
    with torch.no_grad():
        for i, (x,y) in enumerate(dataloader):
            x,y=x.cuda(gpu),y.cuda(gpu)
            logit = net(x,edge_index)
            logits = torch.cat((logits,logit))
            pred = torch.argmax(logit,dim=1)+1
            predictions=torch.cat((predictions,pred))
    return predictions.cpu().numpy(), logits.cpu().numpy()
pred,logits = predict(net,test_loader)
print(pred,logits.shape)

[18.  3.  9. ...  3.  5. 12.] (2959, 49)


In [ ]:
output = pd.read_csv(root_path+"sample.csv")
output['Category']=pred.astype('int')
output.head()
output.to_csv("predictions.csv",index=None)